In [1]:
import os
pygad_loc = '/grid/home/nbourgeois/wip/codon_optimization'
os.chdir(pygad_loc)
import pygad
import numpy as np
from general_functions import *
from metrics import *
import time
import glob
import multiprocessing
from general_functions import *
from metrics import *
import shutil
import sys

Using TensorFlow backend.


# fitness function

In [2]:
cai_on = True
bai_on = True
sps_on = False
cpg_on = True
pas_on = False


In [3]:
cai_w = 1 *cai_on
bai_w = 1 *bai_on
sps_w = 1 *sps_on
cpg_w = 1 *cpg_on
pas_w = 1 *pas_on

In [4]:
all_sols = {}

global total_weight
total_weight = sum([cai_w,bai_w,sps_w,cpg_w,pas_w])
ga_input = '/grid/home/nbourgeois/data/codon_jason/pah.pep.fas'
tissue = 'Liver'
cai_index_loc = os.path.join(pygad_loc,'CoCoPUTs_codon_usage/codon_usage/'+tissue+'.codon.txt')
bai_index_loc = os.path.join(pygad_loc,'CoCoPUTs_codon_usage/bicodon_usage/'+tissue+'.bicodon.txt')

codon_usage_table_loc = os.path.join(pygad_loc,'codon_usage.getex.txt')

def fitness_func(solution, solution_idx):
    
    if not type(solution) is str:
        seq_aa = ''.join([codon_to_int[x] for x in solution])
    else:
        seq_aa = solution
#     print(solution_idx)

    tmp_dict = {}
    
    #Check for redundancy
#     if seq_aa in all_sols.keys():
#         fitness = all_sols[seq_aa]['fitness']

#     else:
    fitness = 0

    if cai_on:
        cai = get_cai(seq_aa, cai_index_loc)
        fitness += cai*cai_w
        tmp_dict['cai'] = cai

    if bai_on:
        bai = get_bai(seq_aa, bai_index_loc)
        fitness += bai*bai_w
        tmp_dict['bai'] = bai

    if cpg_on:
        cpg = get_cpg(seq_aa)
        fitness += cpg*cpg_w
        tmp_dict['cpg'] = cpg

    if sps_on:
        sps = get_sps(seq_aa)
        print('SPS retuned.')

        fitness += sps*sps_w
        tmp_dict['sps'] = sps

    if pas_on:
        pas = get_pas(seq_aa)
        fitness += pas*pas_w
        tmp_dict['pas'] = pas

    fitness = fitness/total_weight
    tmp_dict['fitness'] = fitness
    all_sols[seq_aa] = tmp_dict
        
    
    return tmp_dict

    

# Read Files

In [5]:
cdir = '/data/JiraAnalyses/Misc/MISC-17'
os.chdir(cdir)


In [6]:
fas = glob.glob( '*1k.fa')
fas.append('Translated_Region_PAH.fa')                
fas.append('PAH_codon_random.fa')                
# fas =['Translated_Region_PAH.fa'  ]             
fas = fas[0:2]
fas

['PAH_codon_optimized_bai_1k.fa', 'PAH_codon_optimized_bai-n_1k.fa']

In [7]:
os.chdir('/data/JiraAnalyses/Misc/MISC-17')
fastaIds = []
fastaSeqs = []
for file in fas:
    print(file)
    (fastaIds_t, fastaSeqs_t) = readFasta( file )
    fastaIds.extend(fastaIds_t)
    fastaSeqs.extend(fastaSeqs_t)
    
opts = [x.split('_')[0] for x in fastaIds]
fastaSeqs = [str(x) for x in fastaSeqs]
all_seqs = pd.DataFrame(data = {'Type': opts, 'Seq': fastaSeqs,'SeqID':fastaIds})
all_seqs = all_seqs.set_index('SeqID')


PAH_codon_optimized_bai_1k.fa
PAH_codon_optimized_bai-n_1k.fa


# splicing

In [8]:
%%time

sps_results = predictSites_multi(fastaIds,fastaSeqs,.1)

CPU times: user 1min 57s, sys: 10.6 s, total: 2min 7s
Wall time: 2min 3s


In [9]:
sps_results

{'bai_0': 0.0,
 'bai_1': 0.0,
 'bai_2': 0.0,
 'bai_3': 0.5,
 'bai_4': 0.0,
 'bai_5': 0.5,
 'bai_6': 0.5,
 'bai_7': 0.5,
 'bai_8': 0.0,
 'bai_9': 0.0,
 'bai_10': 0.5,
 'bai_11': 0.0,
 'bai_12': 0.5,
 'bai_13': 0.0,
 'bai_14': 0.0,
 'bai_15': 0.0,
 'bai_16': 0.0,
 'bai_17': 0.0,
 'bai_18': 0.0,
 'bai_19': 0.0,
 'bai_20': 1.0,
 'bai_21': 0.0,
 'bai_22': 0.5,
 'bai_23': 1.0,
 'bai_24': 1.0,
 'bai_25': 0.0,
 'bai_26': 0.0,
 'bai_27': 0.0,
 'bai_28': 0.5,
 'bai_29': 0.5,
 'bai_30': 0.0,
 'bai_31': 0.5,
 'bai_32': 0.0,
 'bai_33': 0.5,
 'bai_34': 0.0,
 'bai_35': 0.5,
 'bai_36': 0.0,
 'bai_37': 0.5,
 'bai_38': 0.0,
 'bai_39': 0.0,
 'bai_40': 0.5,
 'bai_41': 1.0,
 'bai_42': 0.5,
 'bai_43': 0.0,
 'bai_44': 0.5,
 'bai_45': 0.0,
 'bai_46': 0.0,
 'bai_47': 0.0,
 'bai_48': 0.0,
 'bai_49': 0.5,
 'bai_50': 0.0,
 'bai_51': 0.5,
 'bai_52': 0.5,
 'bai_53': 1.0,
 'bai_54': 0.5,
 'bai_55': 0.0,
 'bai_56': 0.5,
 'bai_57': 0.0,
 'bai_58': 0.0,
 'bai_59': 0.5,
 'bai_60': 0.0,
 'bai_61': 1.0,
 'bai_62': 0.0,
 '

In [10]:
%%time

def ps_wrapper(fastaIds,fastaSeqs,threshold,q):
    print('a')
    results = predictSites_multi(fastaIds,fastaSeqs,threshold)
    print('a')

#     q.put(results)
    print('a')

    return None

q = multiprocessing.Queue()  #store the result
pro1 = multiprocessing.Process(target=ps_wrapper, args=(fastaIds,fastaSeqs,.1,q))

pro1.start()
pro1.join()
# results = q.get()


a
CPU times: user 11.7 ms, sys: 110 ms, total: 122 ms
Wall time: 9.99 s


In [1]:
results

NameError: name 'results' is not defined

In [ ]:
from multiprocessing import Process, Manager

def ps_wrapper(fastaIds,fastaSeqs,threshold,q):
    print('a')
    results = predictSites_multi(fastaIds,fastaSeqs,threshold)
    print('a')

    L.append("anything")

    print('a')

    return None

def dothing(L, i):  # the managed list `L` passed explicitly.

with Manager() as manager:
    L = manager.list()  # <-- can be shared between processes.
    p = Process(target=ps_wrapper, args=(fastaIds,fastaSeqs,.1,L))  # Passing the list
    p.start()
    processes.append(p)
    p.join()

    

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/data/software/anaconda3/envs/bfx-gpu/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-16-a9dfae9159c1>", line 1, in <module>
    L
NameError: name 'L' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/data/software/anaconda3/envs/bfx-gpu/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/data/software/anaconda3/envs/bfx-gpu/lib/python3.7/site-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/data/so

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/data/software/anaconda3/envs/bfx-gpu/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-16-a9dfae9159c1>", line 1, in <module>
    L
NameError: name 'L' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/data/software/anaconda3/envs/bfx-gpu/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/data/software/anaconda3/envs/bfx-gpu/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3357, in run_ast_nodes
    if (await self.run_code(code, result,  async_=asy)):
  File "/data/software/an

# poly-A

In [94]:
# cdir = os.getcwd()
# script_dir = os.path.join(pygad_loc,'DeepPASTA_gpu')
# os.chdir(script_dir)
# shutil.rmtree('./tmp')

In [95]:
# def add_fasta(fID,seq):
#     header = '>' + fID
#     return('\n'.join([header,seq,'']))

# os.mkdir('tmp')
# files = []
# size=500
# k=0
# n=0
# i=0
# while i < len(fastaIds):
#     infile = './tmp/tmp{}.fa'.format(k)
#     filestr = ''
#     for j in range(size):
#         if i >= len(fastaIds):
#             break
#         fID = fastaIds[i]
#         seq = fastaSeqs[i]
#         filestr = filestr + add_fasta(fID,seq)
#         files.append(infile)
#         i+=1
#         n+=1
#     with open(infile,'w') as fileo:
#         fileo.write(filestr)       
       

#     k+=1


In [96]:
# def launch_deepPASTA(num):
#     num = str(num)
#     infile = './tmp/tmp{}.fa'.format(num)
#     out_pre='./tmp/tmp{}'.format(num)
#     cmd = 'nice sh ./deep_pasta.sh {infile} {out_pre}'.format(infile=infile, out_pre=out_pre)
#     print(cmd)
#     cmd = cmd.split(' ')
#     process=sp.Popen(cmd)
#     result = process.communicate()
#     print(result)

In [97]:
# import multiprocessing 
# numOfFiles = int(np.ceil((len(fastaIds))/size))
# pool = multiprocessing.Pool(numOfFiles)
# outs = pool.map(launch_deepPASTA, range(numOfFiles))

# filter

## sps

In [98]:
# get max
original_max=0
for key in sps_results.keys():
    opt = key.split('_')[0]
    if opt == 'Original':
        if original_max < sps_results[key]:
            original_max = sps_results[key]
# filter
limit = original_max +.5
print(limit)
filt_fastaIds = []
for key in sps_results.keys():
    opt = key.split('_')[0]
    if opt == 'RandomSeq':
        continue
    if sps_results[key] <= limit:
        filt_fastaIds.append(key)

2.0


In [99]:
filt_seqs = all_seqs.loc[filt_fastaIds]


In [100]:
print('Random' in 'SeqID:RandomSeq_2')

True


### Poor SPS 

In [101]:
# get max
original_min=0
for key in sps_results.keys():
    opt = key.split('_')[0]
    if original_min < sps_results[key]:
        original_min = sps_results[key]
# filter
limit = original_min -.5
filt_fastaIds_poor = []
for key in sps_results.keys():
    opt = key.split('_')[0]
    if sps_results[key] >= limit:
        filt_fastaIds_poor.append(key)

In [102]:
filt_seqs_poor = all_seqs.loc[filt_fastaIds_poor]


In [105]:
filt_seqs_poor['Type'] = 'PoorSplicing'

## Distance

In [55]:
def compute_HammingDistance(X):
    return (X[:, None, :] != X).sum(2)

def check_sol(i,j,k):
    ij_d = distMat[i,j]
    ik_d = distMat[i,k]
    jk_d = distMat[j,k]
    return(ij_d + ik_d + jk_d)

,Type,Seq
SeqID,,
RandomSeq_0,RandomSeq,ATGAGCACAGCCGTGTTAGAAAATCCAGGGCTAGGGCGGAAGCTTT...
RandomSeq_1,RandomSeq,ATGTCTACGGCAGTGTTAGAAAATCCGGGTCTTGGACGTAAACTGT...
RandomSeq_2,RandomSeq,ATGAGCACGGCGGTTCTCGAGAATCCTGGCCTCGGAAGGAAGCTGT...


In [58]:
# sols = {}
# for seq_type in filt_seqs['Type'].unique():
for seq_type in ['cpg-n']:
    if seq_type == 'Original':
        continue
    print(seq_type)
    filt_seqs_sub = filt_seqs.loc[filt_seqs['Type']==seq_type]
    top_seqs = np.array([np.array(list(seq)) for seq in filt_seqs_sub['Seq']])
    #distance_matrix
    distMat = compute_HammingDistance(top_seqs)

    max_dist = 0
    for i in range(len(distMat)-2):
        j=i+1
        while (j < (len(distMat)-1)):
            k=j+1
            while (k < len(distMat)):
                if check_sol(i,j,k) > max_dist:
                    sol = (i,j,k)
                    max_dist = check_sol(i,j,k)
                k+=1
            j+=1
    sols[seq_type] = filt_seqs_sub.iloc[list(sol)].copy()

cpg-n


In [109]:
sols['RandomSeq'] = all_seqs.loc[all_seqs['Type']=='RandomSeq'][0:3]
sols['PoorSplicing'] = filt_seqs_poor
sols

{'cai':         Type                                                Seq
 SeqID                                                          
 cai_2    cai  ATGTCGACGGCTGTCCTGGAGAACCCCGGCCTGGGCCGAAAACTCA...
 cai_358  cai  ATGAGTACCGCTGTGCTGGAGAACCCAGGCCTGGGCAGAAAGCTGA...
 cai_863  cai  ATGAGCACCGCCGTGCTGGAAAACCCAGGCCTCGGCCGCAAGCTGA...,
 'cpg':         Type                                                Seq
 SeqID                                                          
 cpg_150  cpg  ATGAGTACAGCTGTGCTGGAGAATCCAGGGCTGGGCAGAAAACTGT...
 cpg_231  cpg  ATGAGCACAGCTGTCCTAGAGAACCCAGGATTAGGCAGGAAGCTCT...
 cpg_486  cpg  ATGTCAACAGCTGTACTTGAAAATCCAGGGTTGGGGAGGAAACTCT...,
 'bai':         Type                                                Seq
 SeqID                                                          
 bai_515  bai  ATGTCAACCGCCGTTCTGGAGAATCCTGGCCTGGGCAGAAAGCTGT...
 bai_683  bai  ATGAGCACAGCCGTCTTAGAGAACCCTGGCCTGGGCAGAAAGCTGT...
 bai_985  bai  ATGAGCACTGCCGTCCTGGAGAACCCTGGACTGGGCAGAAAGCTGA...,
 

In [147]:
file = ''
for key in sols:
    df = sols[key]
    for row in df.iterrows():
        index = row[0]
        seq = row[1]['Seq']
        fa_header = ">:{0}\n".format(index)
        file = file + fa_header + seq + '\n'


# Add PAS

In [156]:
script_dir = os.path.join(pygad_loc,'DeepPASTA_gpu')
os.chdir(script_dir)

In [150]:
with open('final_seq_prePAS.fa','w') as fileo:
    fileo.write(file)

In [155]:
os.getcwd()

'/grid/home/nbourgeois/data/codon_jason'

ERROR! Session/line number was not unique in database. History logging moved to new session 623


In [152]:
infile = 'final_seq_prePAS.fa'
out_pre= 'final_seq_prePAS'
cmd = 'nice sh ./deep_pasta.sh {infile} {out_pre}'.format(infile=infile, out_pre=out_pre)
print(cmd)
cmd = cmd.split(' ')
process=sp.Popen(cmd)
result = process.communicate()
print(result)

nice sh ./deep_pasta.sh final_seq_prePAS.fa final_seq_prePAS
(None, None)


In [158]:
df_pas = pd.read_csv('final_seq_prePAS.report.txt',sep='\t',header=None)
df_pas
df_pas[2] = df_pas[0].str.split(':',expand=True)[1]
pas_results = df_pas.groupby(2)[1].max().to_dict()
pas_results

{'RandomSeq_0': 0.9991083,
 'RandomSeq_1': 0.95884514,
 'RandomSeq_1208': 0.9742315,
 'RandomSeq_129': 0.9731382,
 'RandomSeq_1737': 0.944178,
 'RandomSeq_2': 0.9542502,
 'RandomSeq_2824': 0.9872716,
 'RandomSeq_4425': 0.99525714,
 'RandomSeq_5226': 0.99223137,
 'bai-n_278': 0.9390886,
 'bai-n_922': 0.9964222,
 'bai-n_965': 0.9935431,
 'bai_515': 0.9600135,
 'bai_683': 0.9628558,
 'bai_985': 0.97097254,
 'cai-n_464': 0.99530435,
 'cai-n_470': 0.999032,
 'cai-n_770': 0.9972291,
 'cai_2': 0.9806903,
 'cai_358': 0.99561775,
 'cai_863': 0.98875964,
 'cpg-n_291': 0.98752344,
 'cpg-n_820': 0.98161,
 'cpg-n_824': 0.9942572,
 'cpg_150': 0.96290445,
 'cpg_231': 0.98681664,
 'cpg_486': 0.99930894}

In [164]:
seq_ids = []
bais = []
cais = []
cpgs = []
spss = []
passs = []
fitnesses = []

for key in sols:
    df = sols[key]
    i=0
    for row in df.iterrows():
        index = row[0]
        rtype = row[1]['Type']
        seq = row[1]['Seq']
        seq_id = rtype+'_'+str(i)
    #     dist_others = sum(distMat[sol,:].T[sol,:][i]) #
        fit_dict = fitness_func(seq,0)
        cai = fit_dict['cai']
        bai = fit_dict['bai']
        cpg = fit_dict['cpg']
        sps = sps_results[index]
        pas = pas_results[index]
    #     pas = fit_dict['pas']
        fitness = fit_dict['fitness']

        fa_header = ">SeqID:{0}_CAI:{1}_BAI:{2}_CpG:{3}_SPS:{4}_".format(seq_id, cai, bai, cpg, sps) + \
                "PAS:{0}_Fitness:{1}\n".format(pas, fitness)
#         fa_header = ">SeqID:{0}\n".format(seq_id)
        i+=1
        print(fa_header)
        file = file + fa_header + seq + '\n'
        seq_ids.append(seq_id) 
        bais.append(bai)
        cais.append(cai)
        cpgs.append(cpg)
        spss.append(sps)
        passs.append(pas)
        fitnesses.append(fitness)

ERROR! Session/line number was not unique in database. History logging moved to new session 631
>SeqID:cai_0_CAI:0.84_BAI:0.496363121830958_CpG:0.883480825958702_SPS:1.5_PAS:0.9806903_Fitness:0.7399479825965534

>SeqID:cai_1_CAI:0.88_BAI:0.553896578525829_CpG:0.8952802359882006_SPS:0.5_PAS:0.99561775_Fitness:0.7763922715046765

>SeqID:cai_2_CAI:0.878_BAI:0.566451030173352_CpG:0.9056047197640118_SPS:0.5_PAS:0.98875964_Fitness:0.783351916645788

>SeqID:cpg_0_CAI:0.666_BAI:0.350035618716926_CpG:1.0_SPS:1.0_PAS:0.96290445_Fitness:0.672011872905642

>SeqID:cpg_1_CAI:0.65_BAI:0.342027493731196_CpG:1.0_SPS:2.0_PAS:0.98681664_Fitness:0.6640091645770654

>SeqID:cpg_2_CAI:0.681_BAI:0.376565601576856_CpG:1.0_SPS:2.0_PAS:0.99930894_Fitness:0.6858552005256188

>SeqID:bai_0_CAI:0.879_BAI:0.713074359867023_CpG:0.9247787610619469_SPS:1.0_PAS:0.9600135_Fitness:0.8389510403096567

>SeqID:bai_1_CAI:0.886_BAI:0.711613172265529_CpG:0.9292035398230089_SPS:0.0_PAS:0.9628558_Fitness:0.8422722373628458

>SeqID

In [173]:
all_df = pd.DataFrame(data={'SeqID':seq_ids,'CAI':cais,'BAI':bais,'CpG':cpgs,'SPS':spss,'PAS':passs,'Fitness':fitnesses})
all_df = all_df.set_index('SeqID')
all_df.to_csv('meta_table.csv')

In [160]:
os.chdir(cdir)
with open('final_seq.fa','w') as fileo:
    fileo.write(file)